In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23964287
paper_name = 'gaytan_vulpe_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [11]:
original_data = pd.read_excel('raw_data/DataSheet2.xlsx', sheet_name='mutants with altered growth in ', skiprows=9)

In [12]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 264 x 4


In [13]:
original_data.head()

,ORF,Name,Log2 1% DMSO,Description
0,YIL162W,SUC2,-4.537100,"Invertase, sucrose hydrolyzing enzyme; a secre..."
1,YHR010W,RPL27A,-2.450592,Protein component of the large (60S) ribosomal...
2,YDR083W,RRP8,-2.413252,"Nucleolar protein involved in rRNA processing,..."
3,YNL051W,COG5,-2.384488,Component of the conserved oligomeric Golgi co...
4,YER156C,-,-2.313167,Putative protein of unknown function; interact...


In [14]:
original_data['orf'] = original_data['ORF'].astype(str)

In [15]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [16]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [17]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Name, Log2 1% DMSO, Description, orf]
Index: []


In [18]:
original_data['data'] = original_data['Log2 1% DMSO']

In [19]:
original_data.set_index('orf', inplace=True)

In [20]:
original_data = original_data[['data']].copy()

In [21]:
original_data = original_data.groupby(original_data.index).mean()

In [22]:
original_data.shape

(264, 1)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [196]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,196
data_type,value
orf,
YAL015C,-1.050390
YAL042C-A,1.643904
YAR031W,1.602729
YAR042W,1.500711
YBL094C,1.636652


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,196
,data_type,value
gene_id,orf,
13,YAL015C,-1.050390
1865,YAL042C-A,1.643904
75,YAR031W,1.602729
78,YAR042W,1.500711
182,YBL094C,1.636652


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id              196          
data_type             value    valuez
gene_id orf                          
13      YAL015C   -1.050390 -2.269437
1865    YAL042C-A  1.643904  3.500224
75      YAR031W    1.602729  3.412050
78      YAR042W    1.500711  3.193584
182     YBL094C    1.636652  3.484694

# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [33]:
from IO.save_data_to_db3 import *

In [34]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 23964287...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.73it/s]

Updating the data_modified_on field...
